#Preprocessing

In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

import numpy as np
import pandas as pd

header = ['user_id', 'item_id', 'rating', 'timestamp']
drivePrefix = '/content/gdrive/My Drive/Tubes_1301164036/'
driveSuffix = 'ml-latest-small/ratings.csv'
df_rating = pd.read_csv(drivePrefix + driveSuffix, sep=',', names =header, skiprows=1)

n_users = df_rating.user_id.unique().shape[0]
n_items = df_rating.item_id.unique().shape[0]
print('Number of users = ' + str(n_users) + ' | number of movies = ' + str(n_items))
print(df_rating)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
Number of users = 610 | number of movies = 9724
        user_id  item_id  rating   timestamp
0             1        1     4.0   964982703
1             1        3     4.0   964981247
2             1        6     4.0   964982224
3             1       47     5.0   964983815
4             1       50     5.0   964982931
...         ...      ...     ...         ...
100831      610   166534     4.0  1493848402
100832      610   168248     5.0  1493850091
100833    

In [0]:
pd.DataFrame(df_rating)

,user_id,item_id,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [0]:
R = np.nan_to_num(np.array(df_rating.pivot(index='user_id', columns='item_id', values='rating')))
R1 = df_rating.pivot(index='user_id', columns='item_id', values='rating').fillna(0)

In [0]:
pd.DataFrame(R)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,9684,9685,9686,9687,9688,9689,9690,9691,9692,9693,9694,9695,9696,9697,9698,9699,9700,9701,9702,9703,9704,9705,9706,9707,9708,9709,9710,9711,9712,9713,9714,9715,9716,9717,9718,9719,9720,9721,9722,9723
0,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605,2.5,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,2.5,0.0,0.0,0.0,3.5,0.0,4.0,4.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,4.5,0.0,0.0,4.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
606,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
607,2.5,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,4.5,0.0,0.0,2.0,0.0,3.5,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.5,3.5,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
608,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
R1

item_id,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,36,38,39,40,41,42,43,...,185135,185435,185473,185585,186587,187031,187541,187593,187595,187717,188189,188301,188675,188751,188797,188833,189043,189111,189333,189381,189547,189713,190183,190207,190209,190213,190215,190219,190221,191005,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.5,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,2.5,0.0,0.0,0.0,3.5,0.0,4.0,4.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,4.5,0.0,0.0,4.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
607,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
608,2.5,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,4.5,0.0,0.0,2.0,0.0,3.5,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.5,3.5,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Matrix Factorisation

In [0]:
#!/usr/bin/python
#
# Created by Albert Au Yeung (2010)
#
# An implementation of matrix factorization
#
try:
    from numpy import savetxt
except:
    print ("This implementation requires the numpy module.")
    exit(0)

###############################################################################

"""
@INPUT:
    R     : a matrix to be factorized, dimension N x M
    P     : an initial matrix of dimension N x K
    Q     : an initial matrix of dimension M x K
    K     : the number of latent features
    steps : the maximum number of steps to perform the optimisation
    alpha : the learning rate
    beta  : the regularization parameter
@OUTPUT:
    the final matrices P and Q
"""
def gl_average(matrix):
  hasil,nol = 0,0
  for i in matrix:
      for j in i:
        if j != 0:
          nol+=1
          hasil = j + hasil
  return (hasil/nol)

def bias (matrix):
  userbias,itembias= [],[]
  for i in range(len(matrix)):
    totalnol = np.count_nonzero(matrix[i])
    if totalnol == 0:
      userbias.append(1 - globalaverage)
    else:
      userbias.append((matrix[i].sum()/np.count_nonzero(matrix[i])) - globalaverage)

  for i in range(len(matrix[0])):
    totalnol = np.count_nonzero(matrix[:,i])
    if totalnol == 0:
      itembias.append(1 - globalaverage)
    else:
      itembias.append((matrix[:,i].sum()/np.count_nonzero(matrix[:,i])) - globalaverage)

  return userbias,itembias
def matrix_factorization(matrix, P, Q, K, steps=3, alpha=0.002, beta=0.01):
    Q = Q.T
    for step in range(steps):
        for i in range(len(matrix)):
            for j in range(len(matrix[i])):
                if matrix[i][j] > 0:
                    eij = matrix[i][j] - globalaverage - userbias[i] - itembias[j] - np.dot(P[i,:],Q[:,j])
                    for k in range(K):
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])
        eR = np.dot(P,Q)
        e = 0
        for i in range(len(matrix)):
            for j in range(len(matrix[i])):
                if matrix[i][j] > 0:
                    e = e + pow(matrix[i][j] - globalaverage - userbias[i] - itembias[j] - np.dot(P[i,:],Q[:,j]), 2)
                    for k in range(K):
                        e = e + (beta/2) * ( pow(P[i][k],2) + pow(Q[k][j],2) )
        if e < 0.001:
            break
    return P, Q.T


###############################################################################

if __name__ == "__main__":
    

    N = len(R)
    M = len(R[0])
    K = 1

In [0]:
np.random.seed(30)
globalaverage= gl_average(R)
userbias,itembias = bias(R)
P = np.random.rand(N,K)
Q = np.random.rand(M,K)
nP, nQ = matrix_factorization(R, P, Q, K)

In [0]:
hasil = np.dot(nP, nQ.T)+globalaverage+itembias

In [0]:
for x in range(len(hasil)):
      for y in range(len(hasil[x])):
        hasil[x][y]+=userbias[x]

In [0]:
 pd.DataFrame(hasil)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,9684,9685,9686,9687,9688,9689,9690,9691,9692,9693,9694,9695,9696,9697,9698,9699,9700,9701,9702,9703,9704,9705,9706,9707,9708,9709,9710,9711,9712,9713,9714,9715,9716,9717,9718,9719,9720,9721,9722,9723
0,4.895731,4.399710,4.348350,3.326720,4.095282,4.912328,4.068394,3.893184,4.200588,4.466030,4.648386,3.539877,4.107874,4.946802,4.080344,4.964444,4.811728,4.749101,3.804234,3.394035,4.393063,4.317947,4.233349,4.224337,4.549117,4.422592,4.409184,5.257045,4.893146,4.050364,4.240299,4.926860,4.704060,4.719928,3.192721,4.305152,5.289318,4.778730,4.115442,4.276649,...,5.620993,3.875604,4.138056,3.864197,3.941319,4.254132,3.995701,5.005741,5.023842,5.984157,5.643791,4.540045,4.564517,5.440901,4.975750,5.632847,3.458181,4.138224,4.677541,3.482490,2.142178,3.396217,4.450431,2.506264,4.903561,1.921659,2.446515,2.116389,1.997596,5.462423,4.451235,3.917128,5.002193,4.947744,4.473732,4.936313,4.403043,4.569509,4.494614,4.900481
1,4.476530,3.980578,3.928012,2.907572,3.675592,4.493213,3.650107,3.473550,3.780382,4.046879,4.229164,3.119240,3.688592,4.526218,3.660090,4.544617,4.391921,4.329159,3.384014,2.975640,3.974622,3.897540,3.812816,3.803894,4.130422,4.003912,3.988975,4.837296,4.474891,3.630410,3.820287,4.507969,4.284092,4.301632,2.772593,3.885581,4.869474,4.358492,3.694839,3.856307,...,5.202828,3.457393,3.717227,3.443606,3.522453,3.834639,3.576292,4.584985,4.603155,5.564863,5.222904,4.121856,4.144421,5.022038,4.556540,5.212070,3.039147,3.717393,4.258811,3.063212,1.721308,2.977800,4.031473,2.086749,4.485071,1.502988,2.027596,1.695775,1.578168,5.043345,4.032269,3.498503,4.582719,4.528814,4.054542,4.517496,3.984558,4.149364,4.075215,4.482021
2,3.060938,2.558905,2.612688,1.487383,2.303169,3.070095,2.153909,2.096137,2.453488,2.626902,2.815471,1.830396,2.280105,3.232660,2.337382,3.184305,3.029843,2.978956,2.058328,1.488970,2.492040,2.588367,2.514753,2.497900,2.670225,2.542373,2.662312,3.470081,2.975859,2.281317,2.476232,3.065103,2.936189,2.806226,1.438776,2.502623,3.510660,3.034363,2.403018,2.541347,...,3.695880,1.954502,2.445308,2.150689,2.077396,2.444860,2.179093,3.306628,3.318727,4.157506,3.956032,2.617007,2.807784,3.576613,3.141783,3.935566,1.608929,2.445622,2.801628,1.654387,0.453016,1.493051,2.594435,0.698846,3.006785,0.040629,0.587111,0.404788,0.182638,3.616860,2.595939,2.032156,3.191234,3.089411,2.638010,3.068033,2.505815,2.817120,2.677060,3.001024
3,3.906098,3.421310,3.173477,2.345580,3.025897,3.936598,3.227677,2.833017,3.047090,3.484499,3.655247,2.316083,3.105116,3.731715,2.919113,3.872685,3.723230,3.638667,2.648503,2.535716,3.527190,3.131709,3.026586,3.032226,3.641889,3.517843,3.255259,4.178037,4.057665,2.937876,3.118504,3.987615,3.589374,3.877750,2.052013,3.255230,4.194790,3.620124,2.897147,3.101103,...,4.800136,3.047251,2.882994,2.647926,3.006122,3.216811,2.972088,3.762572,3.791889,4.979312,4.379405,3.715311,3.429019,4.506384,3.984574,4.386254,2.495569,2.882889,3.764746,2.480356,0.880414,2.534350,3.500420,1.465477,4.029754,1.018427,1.502871,0.896554,0.970901,4.492915,3.499917,3.021263,3.968024,4.002101,3.485837,4.009256,3.530078,3.425894,3.472768,4.031682
4,4.373995,3.864889,4.042385,2.795092,3.666443,4.374398,3.373198,3.453607,3.869726,3.934858,4.130736,3.290898,3.601427,4.687679,3.758489,4.561668,4.405152,4.368078,3.475972,2.719342,3.727169,4.025220,3.964530,3.938451,3.931394,3.801981,4.078818,4.839415,4.191851,3.671732,3.872507,4.346432,4.327988,4.026435,2.846961,3.853642,4.889767,4.453818,3.860058,3.971468,...,4.902664,3.166006,3.925498,3.606454,3.356176,3.787944,3.513546,4.779330,4.784366,5.480141,5.442093,3.826232,4.212688,4.854965,4.455812,5.410424,2.904972,3.925984,4.066302,2.975315,1.937426,2.725657,3.882543,2.044114,4.246910,1.299282,1.871210,1.862796,1.519031,4.917245,3.884870,3.286170,4.532333,4.374768,3.949973,4.341687,3.745410,4.227135,4.010400,4.237996
...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [0]:
hasil = pd.DataFrame(hasil,index=df_rating.user_id.unique(),columns=np.sort(df_rating.item_id.unique()))

In [0]:
hasil

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,36,38,39,40,41,42,43,...,185135,185435,185473,185585,186587,187031,187541,187593,187595,187717,188189,188301,188675,188751,188797,188833,189043,189111,189333,189381,189547,189713,190183,190207,190209,190213,190215,190219,190221,191005,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
1,4.895731,4.399710,4.348350,3.326720,4.095282,4.912328,4.068394,3.893184,4.200588,4.466030,4.648386,3.539877,4.107874,4.946802,4.080344,4.964444,4.811728,4.749101,3.804234,3.394035,4.393063,4.317947,4.233349,4.224337,4.549117,4.422592,4.409184,5.257045,4.893146,4.050364,4.240299,4.926860,4.704060,4.719928,3.192721,4.305152,5.289318,4.778730,4.115442,4.276649,...,5.620993,3.875604,4.138056,3.864197,3.941319,4.254132,3.995701,5.005741,5.023842,5.984157,5.643791,4.540045,4.564517,5.440901,4.975750,5.632847,3.458181,4.138224,4.677541,3.482490,2.142178,3.396217,4.450431,2.506264,4.903561,1.921659,2.446515,2.116389,1.997596,5.462423,4.451235,3.917128,5.002193,4.947744,4.473732,4.936313,4.403043,4.569509,4.494614,4.900481
2,4.476530,3.980578,3.928012,2.907572,3.675592,4.493213,3.650107,3.473550,3.780382,4.046879,4.229164,3.119240,3.688592,4.526218,3.660090,4.544617,4.391921,4.329159,3.384014,2.975640,3.974622,3.897540,3.812816,3.803894,4.130422,4.003912,3.988975,4.837296,4.474891,3.630410,3.820287,4.507969,4.284092,4.301632,2.772593,3.885581,4.869474,4.358492,3.694839,3.856307,...,5.202828,3.457393,3.717227,3.443606,3.522453,3.834639,3.576292,4.584985,4.603155,5.564863,5.222904,4.121856,4.144421,5.022038,4.556540,5.212070,3.039147,3.717393,4.258811,3.063212,1.721308,2.977800,4.031473,2.086749,4.485071,1.502988,2.027596,1.695775,1.578168,5.043345,4.032269,3.498503,4.582719,4.528814,4.054542,4.517496,3.984558,4.149364,4.075215,4.482021
3,3.060938,2.558905,2.612688,1.487383,2.303169,3.070095,2.153909,2.096137,2.453488,2.626902,2.815471,1.830396,2.280105,3.232660,2.337382,3.184305,3.029843,2.978956,2.058328,1.488970,2.492040,2.588367,2.514753,2.497900,2.670225,2.542373,2.662312,3.470081,2.975859,2.281317,2.476232,3.065103,2.936189,2.806226,1.438776,2.502623,3.510660,3.034363,2.403018,2.541347,...,3.695880,1.954502,2.445308,2.150689,2.077396,2.444860,2.179093,3.306628,3.318727,4.157506,3.956032,2.617007,2.807784,3.576613,3.141783,3.935566,1.608929,2.445622,2.801628,1.654387,0.453016,1.493051,2.594435,0.698846,3.006785,0.040629,0.587111,0.404788,0.182638,3.616860,2.595939,2.032156,3.191234,3.089411,2.638010,3.068033,2.505815,2.817120,2.677060,3.001024
4,3.906098,3.421310,3.173477,2.345580,3.025897,3.936598,3.227677,2.833017,3.047090,3.484499,3.655247,2.316083,3.105116,3.731715,2.919113,3.872685,3.723230,3.638667,2.648503,2.535716,3.527190,3.131709,3.026586,3.032226,3.641889,3.517843,3.255259,4.178037,4.057665,2.937876,3.118504,3.987615,3.589374,3.877750,2.052013,3.255230,4.194790,3.620124,2.897147,3.101103,...,4.800136,3.047251,2.882994,2.647926,3.006122,3.216811,2.972088,3.762572,3.791889,4.979312,4.379405,3.715311,3.429019,4.506384,3.984574,4.386254,2.495569,2.882889,3.764746,2.480356,0.880414,2.534350,3.500420,1.465477,4.029754,1.018427,1.502871,0.896554,0.970901,4.492915,3.499917,3.021263,3.968024,4.002101,3.485837,4.009256,3.530078,3.425894,3.472768,4.031682
5,4.373995,3.864889,4.042385,2.795092,3.666443,4.374398,3.373198,3.453607,3.869726,3.934858,4.130736,3.290898,3.601427,4.687679,3.758489,4.561668,4.405152,4.368078,3.475972,2.719342,3.727169,4.025220,3.964530,3.938451,3.931394,3.801981,4.078818,4.839415,4.191851,3.671732,3.872507,4.346432,4.327988,4.026435,2.846961,3.853642,4.889767,4.453818,3.860058,3.971468,...,4.902664,3.166006,3.925498,3.606454,3.356176,3.787944,3.513546,4.779330,4.784366,5.480141,5.442093,3.826232,4.212688,4.854965,4.455812,5.410424,2.904972,3.925984,4.066302,2.975315,1.937426,2.725657,3.882543,2.044114,4.246910,1.299282,1.871210,1.862796,1.519031,4.917245,3.884870,3.286170,4.532333,4.374768,3.949973,4.341687,3.745410,4

In [0]:
recommend,rating = [],[]
for data in range(len(hasil)):
  recommend.append(hasil.iloc[data].sort_values(ascending=False).keys()[:1000])
  rating.append(np.array(hasil.iloc[data].sort_values(ascending=False)[:1000])) 

In [0]:
pd.DataFrame(np.array(recommend))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999
0,166183,158882,1349,136556,69211,5537,138835,27704,93008,2075,6192,108078,173355,146662,25947,73822,3473,26147,120138,94810,142444,165959,47736,164367,25887,146684,157775,130970,626,124851,173619,467,136341,27320,2969,172637,3940,53355,136850,5746,...,1198,122926,1136,31658,8207,65642,147374,51705,6184,57669,3675,79357,4967,2585,1096,912,2333,1103,3415,68194,47491,7706,5039,1945,1249,1277,69481,173197,3469,4278,1437,92475,2138,6612,3030,109968,2019,3089,6945,80139
1,166183,158882,1349,136556,69211,5537,138835,27704,93008,2075,6192,108078,173355,146662,25947,73822,3473,26147,120138,94810,142444,165959,47736,164367,25887,146684,157775,130970,626,124851,173619,467,136341,27320,2969,172637,3940,53355,136850,5746,...,6350,122926,65642,1136,147374,51705,8207,31658,6184,57669,2585,3675,4967,912,79357,68194,2333,1096,3415,47491,1103,7706,1945,5039,173197,1249,1277,1437,69481,3469,4278,92475,6612,3030,109968,2019,2138,6945,3089,80139
2,166183,158882,1349,136556,69211,5537,138835,27704,93008,2075,6192,108078,173355,146662,25947,73822,3473,26147,120138,94810,142444,165959,47736,164367,25887,146684,157775,130970,626,124851,173619,467,136341,27320,2969,172637,3940,53355,136850,5746,...,1103,152173,27156,1423,506,583,8011,34359,6195,1477,149830,1900,55444,3019,106,757,154975,80083,8167,1936,961,87287,106889,4759,84553,102194,3182,167570,128908,38198,6375,1121,73344,27834,7055,26444,1990,170357,37444,6775
3,6611,82744,3086,8911,83969,27523,142020,143559,95311,6983,95149,26587,60737,1310,150554,147326,100906,27751,3567,76091,8238,3939,100556,3851,113829,108795,134796,496,71268,633,4116,27373,151769,6201,5513,96935,26849,92494,131130,172909,...,3983,1225,4967,26554,4499,2019,48516,3653,3090,55442,2810,82041,5279,2905,1201,1797,56782,1945,4160,1203,4444,3675,26547,2329,8014,113252,6776,3095,906,800,47404,115569,173925,2937,27397,104069,3246,1283,4142,168248
4,166183,158882,1349,136556,69211,5537,138835,27704,93008,2075,6192,108078,173355,146662,25947,73822,3473,26147,120138,94810,142444,165959,47736,164367,25887,146684,157775,130970,626,124851,173619,467,136341,27320,2969,172637,3940,53355,136850,5746,...,6301,48649,25773,5322,4765,105504,3403,8142,71302,2632,4083,4193,27708,4422,69720,2609,48342,3019,4537,173209,145418,26258,7459,49688,668,8577,109971,153408,6808,7766,4660,1144,3783,7382,2924,8645,1949,65738,142598,130976
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605,6611,82744,3086,8911,83969,27523,142020,143559,95311,6983,95149,26587,60737,1310,150554,147326,100906,27751,3567,76091,8238,3939,100556,3851,113829,108795,134796,496,71268,633,4116,27373,151769,6201,5513,96935,26849,92494,131130,172909,...,8838,47404,173925,27397,38095,95654,2329,4142,2810,4967,3074,2019,1225,1419,6963,87234,5120,166946,2565,55442,1945,108727,116897,3983,5425,3675,5379,31437,1210,180497,3653,3090,71462,8724,1283,6530,159817,66943,99764,1208
606,166183,158882,1349,136556,69211,5537,138835,27704,93008,2075,6192,108078,173355,146662,25947,73822,3473,26147,120138,94810,142444,165959,47736,164367,25887,146684,157775,130970,626,124851,173619,467,136341,27320,2969,172637,3940,53355,136850,5746,...,49265,3019,5250,81831,7236,5741,97188,152173,1423,27834,583,8011,34359,6195,3068,1477,1900,73344,106,757,154975,80083,8167,1936,961,87287,106889,171763,4759,84553,58559,3182,167570,128908,38198,6375,1121,1221,58425,2333
607,166183,158882,1349,136556,69211,5537,138835,27704,93008,2075,6192,108078,173355,146662,25947,73822,3473,

In [0]:
pd.DataFrame(rating)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999
0,6.146790,6.145754,6.143928,6.143813,6.143658,6.143114,6.142111,6.141512,6.140407,6.140250,6.133948,6.131897,6.129637,6.128902,6.128835,6.128105,6.127088,6.126087,6.124145,6.124037,6.123866,6.122790,6.119446,6.118913,6.118729,6.118698,6.118476,6.118438,6.117638,6.117585,6.116845,6.116607,6.114859,6.113539,6.112680,6.112292,6.111098,6.110103,6.109472,6.109459,...,5.195376,5.194054,5.193217,5.192924,5.192914,5.192461,5.192276,5.191352,5.190709,5.184399,5.183472,5.183386,5.183170,5.183136,5.182760,5.182495,5.182191,5.181397,5.181364,5.181296,5.180788,5.180624,5.179218,5.178912,5.178694,5.177992,5.177784,5.177437,5.177275,5.176385,5.175964,5.173820,5.173285,5.173255,5.172966,5.172710,5.171923,5.171541,5.171111,5.170582
1,5.725873,5.724848,5.723040,5.722926,5.722773,5.722234,5.721241,5.720648,5.719554,5.719399,5.713160,5.711129,5.708891,5.708163,5.708097,5.707375,5.706368,5.705377,5.703454,5.703347,5.703178,5.702113,5.698802,5.698274,5.698093,5.698061,5.697842,5.697804,5.697012,5.696960,5.696227,5.695991,5.694261,5.692954,5.692104,5.691719,5.690537,5.689552,5.688928,5.688915,...,4.775279,4.774224,4.773583,4.773451,4.772569,4.772485,4.772369,4.772295,4.771848,4.764688,4.764351,4.764060,4.763809,4.763617,4.763529,4.762529,4.762168,4.761983,4.761765,4.761195,4.761132,4.760617,4.759545,4.758809,4.758709,4.758172,4.758011,4.757251,4.757145,4.756885,4.756836,4.755127,4.754569,4.754091,4.754029,4.752626,4.752602,4.752446,4.751625,4.751091
2,4.461640,4.459703,4.456288,4.456073,4.455784,4.454767,4.452890,4.451770,4.449704,4.449410,4.437625,4.433790,4.429562,4.428188,4.428063,4.426698,4.424796,4.422924,4.419293,4.419090,4.418771,4.416760,4.410505,4.409508,4.409165,4.409106,4.408691,4.408620,4.407124,4.407025,4.405642,4.405196,4.401927,4.399458,4.397852,4.397126,4.394893,4.393033,4.391853,4.391829,...,3.439350,3.439186,3.438878,3.438830,3.438142,3.437904,3.437060,3.436635,3.436517,3.436212,3.436142,3.435974,3.434666,3.434516,3.433778,3.433428,3.431588,3.431144,3.430842,3.430837,3.430831,3.430540,3.430174,3.428618,3.428532,3.428502,3.427984,3.427967,3.427348,3.426376,3.426213,3.426181,3.424511,3.424066,3.423920,3.423798,3.423604,3.423536,3.422535,3.421275
3,5.053077,5.052871,5.052639,5.052468,5.051869,5.051535,5.051002,5.049321,5.048267,5.047354,5.046984,5.046747,5.046568,5.046274,5.045966,5.045789,5.043758,5.042779,5.042340,5.042116,5.041771,5.041535,5.041286,5.040880,5.040354,5.040071,5.039969,5.039525,5.038919,5.038911,5.038542,5.038397,5.037902,5.037764,5.036714,5.035661,5.035067,5.034303,5.034015,5.033656,...,4.168503,4.167754,4.167424,4.167046,4.166688,4.166646,4.165239,4.165219,4.165017,4.164561,4.164476,4.164273,4.164086,4.163985,4.163329,4.163228,4.162619,4.162272,4.161786,4.161705,4.159881,4.159418,4.159357,4.158338,4.157191,4.156330,4.155514,4.155059,4.152977,4.152603,4.151721,4.150748,4.150424,4.150289,4.149440,4.148312,4.148084,4.147200,4.145406,4.145352
4,5.950771,5.947774,5.942490,5.942157,5.941709,5.940135,5.937231,5.935497,5.932301,5.931847,5.913610,5.907676,5.901133,5.899006,5.898813,5.896701,5.893758,5.890861,5.885242,5.884928,5.884434,5.881321,5.871643,5.870100,5.869569,5.869478,5.868836,5.868726,5.866411,5.866258,5.864117,5.863427,5.858369,5.854549,5.852063,5.850940,5.847484,5.844606,5.842780,5.842742,...,4.831776,4.831604,4.831500,4.831333,4.831091,4.830678,4.830431,4.830258,4.830020,4.829934,4.829155,4.828364,4.828357,4.828030,4.827924,4.827129,4.826926,4.826683,4.826123,4.825882,4.825416,4.825400,4.824931,4.824850,4.824668,4.824548,4.823969,4.823433,4.823180,4.823068,4.822676,4.821384,4.820345,4.820334,4.819696,4.819585,4.819294,4.819049,4.818858,4.818337
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

# Fix Cluster Movie Genre

In [0]:
header = ['movieId', 'title', 'genre']
drivePrefix = '/content/gdrive/My Drive/Tubes_1301164036/'
driveSuffix = 'ml-latest-small/movies.csv'
df_genre = pd.read_csv(drivePrefix + driveSuffix, sep=',', names =header, skiprows=1)

# n_users = df.user_id.unique().shape[0]
n_items = df_genre.movieId.unique().shape[0]
print('number of movies = ' + str(n_items))
print(df_genre)

number of movies = 9742
      movieId  ...                                        genre
0           1  ...  Adventure|Animation|Children|Comedy|Fantasy
1           2  ...                   Adventure|Children|Fantasy
2           3  ...                               Comedy|Romance
3           4  ...                         Comedy|Drama|Romance
4           5  ...                                       Comedy
...       ...  ...                                          ...
9737   193581  ...              Action|Animation|Comedy|Fantasy
9738   193583  ...                     Animation|Comedy|Fantasy
9739   193585  ...                                        Drama
9740   193587  ...                             Action|Animation
9741   193609  ...                                       Comedy

[9742 rows x 3 columns]


In [0]:
pd.Series(df_genre['title'])

0                                Toy Story (1995)
1                                  Jumanji (1995)
2                         Grumpier Old Men (1995)
3                        Waiting to Exhale (1995)
4              Father of the Bride Part II (1995)
                          ...                    
9737    Black Butler: Book of the Atlantic (2017)
9738                 No Game No Life: Zero (2017)
9739                                 Flint (2017)
9740          Bungo Stray Dogs: Dead Apple (2018)
9741          Andrew Dice Clay: Dice Rules (1991)
Name: title, Length: 9742, dtype: object

In [0]:
temp,vec = [],[]
for i in df_genre.genre:
  data = i.split('|')
  vec.append(data)
  for j in (data):
    temp.append(j)

In [0]:
temp = pd.unique(temp)

In [0]:
vec_movie = pd.DataFrame(index = df_genre.genre.keys(), columns =temp)

In [0]:
for i in range (len(vec)):
   vec_movie.loc[i, vec[i]] = 1

In [0]:
vec_movie = vec_movie.fillna(0)

In [0]:
vec_movie

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9737,0,1,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
9738,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9739,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
9740,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
from sklearn.cluster import KMeans
kmeans = KMeans( random_state=0).fit(vec_movie)

In [0]:
kmeans.labels_

array([7, 7, 1, ..., 4, 6, 3], dtype=int32)

In [0]:
pd.unique(kmeans.labels_)

array([7, 1, 3, 2, 6, 4, 5, 0], dtype=int32)

In [0]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = [0 for i in range(8)]
for i in range(8):  
  idx2 = np.where(kmeans.labels_==i)
  cosine_sim[i] = cosine_similarity(vec_movie.iloc[idx2],vec_movie.iloc[idx2])
  cosine_sim[i] = pd.DataFrame(cosine_sim[i], index=idx2[0],columns=idx2[0])

In [0]:
cosine_sim[1][2].sort_values(ascending = False)

4390    1.000000
4488    1.000000
4688    1.000000
4666    1.000000
4614    1.000000
          ...   
3713    0.316228
7078    0.316228
618     0.316228
4833    0.316228
1122    0.288675
Name: 2, Length: 1449, dtype: float64

In [0]:
recommend_cb = []
for data in range(len(R1)) :
  temp = np.where(R1.iloc[data]>=4.0)
  temp = pd.Series(R1.iloc[data]).keys()[temp]
  predict_recommend = []
  for film in temp:
    idx = np.where(df_genre.movieId==film)[0] 
    top_5 = pd.Series(cosine_sim[kmeans.labels_[idx[0]]][idx[0]]).sort_values(ascending = False).keys()[0:5]
    predict_recommend.append(np.array(df_genre.movieId[top_5]))
  temp2 = np.unique(np.array(predict_recommend).flatten())
  recommend_cb.append(temp2)

In [0]:
 pd.DataFrame(recommend_cb)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,1408,1409,1410,1411,1412,1413,1414,1415,1416,1417,1418,1419,1420,1421,1422,1423,1424,1425,1426,1427,1428,1429,1430,1431,1432,1433,1434,1435,1436,1437,1438,1439,1440,1441,1442,1443,1444,1445,1446,1447
0,1.0,15.0,50.0,95.0,101.0,157.0,163.0,164.0,198.0,256.0,262.0,265.0,296.0,346.0,356.0,362.0,367.0,434.0,442.0,473.0,485.0,519.0,543.0,546.0,551.0,552.0,553.0,588.0,589.0,590.0,594.0,661.0,671.0,673.0,674.0,709.0,737.0,761.0,780.0,836.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,198.0,780.0,1127.0,1129.0,1271.0,1544.0,1598.0,1684.0,1858.0,2170.0,2370.0,2505.0,2905.0,3208.0,3578.0,4065.0,4262.0,4410.0,4903.0,4928.0,4959.0,4970.0,5008.0,5010.0,5248.0,5867.0,6287.0,6313.0,6315.0,6320.0,6333.0,6708.0,8045.0,8530.0,8533.0,8665.0,8912.0,8921.0,26736.0,27022.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9.0,589.0,606.0,611.0,737.0,780.0,849.0,1037.0,1127.0,1129.0,1275.0,1544.0,1882.0,2196.0,2311.0,2373.0,2455.0,2456.0,2606.0,2614.0,2699.0,2851.0,3190.0,3283.0,3444.0,3576.0,3833.0,4443.0,4552.0,4553.0,5039.0,5128.0,5189.0,5219.0,5356.0,5468.0,5746.0,5764.0,5797.0,5919.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,50.0,155.0,161.0,168.0,256.0,265.0,296.0,319.0,340.0,368.0,432.0,442.0,492.0,548.0,551.0,552.0,588.0,648.0,671.0,709.0,714.0,724.0,736.0,761.0,800.0,907.0,908.0,914.0,919.0,923.0,930.0,931.0,998.0,1009.0,1025.0,1028.0,1032.0,1034.0,1049.0,1055.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,50.0,241.0,262.0,296.0,367.0,551.0,588.0,590.0,594.0,595.0,709.0,839.0,1022.0,1025.0,1029.0,1034.0,1055.0,1178.0,1282.0,1438.0,1547.0,1566.0,1689.0,2087.0,2092.0,2110.0,2112.0,2243.0,2262.0,2294.0,2353.0,2579.0,2723.0,3034.0,3147.0,3266.0,3494.0,4011.0,4242.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605,1.0,29.0,32.0,50.0,70.0,95.0,155.0,157.0,159.0,164.0,168.0,198.0,212.0,241.0,256.0,262.0,265.0,296.0,329.0,340.0,346.0,356.0,389.0,427.0,434.0,442.0,451.0,459.0,473.0,519.0,548.0,551.0,552.0,553.0,587.0,589.0,590.0,648.0,671.0,709.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
606,1.0,95.0,150.0,188.0,241.0,262.0,377.0,380.0,382.0,434.0,517.0,553.0,589.0,724.0,736.0,737.0,761.0,780.0,786.0,836.0,839.0,908.0,919.0,940.0,1024.0,1032.0,1049.0,1055.0,1097.0,1101.0,1127.0,1129.0,1178.0,1198.0,1209.0,1249.0,1304.0,1335.0,1336.0,1337.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,1.0,9.0,32.0,50.0,95.0,157.0,159.0,164.0,168.0,183.0,188.0,198.0,212.0,256.0,265.0,296.0,319.0,329.0,353.0,434.0,442.0,473.0,485.0,548.0,552.0,588.0,589.0,610.0,611.0,671.0,673.0,674.0,680.0,709.0,737.0,741.0,761.0,780.0,786.0,836.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,95.0

In [0]:
for i in recommend_cb[0][:5]:
  print(df_genre.loc[df_genre['movieId'] == i].title)

0    Toy Story (1995)
Name: title, dtype: object
14    Cutthroat Island (1995)
Name: title, dtype: object
46    Usual Suspects, The (1995)
Name: title, dtype: object
84    Broken Arrow (1996)
Name: title, dtype: object
89    Bottle Rocket (1996)
Name: title, dtype: object


# Cluster Tags

In [0]:
drivePrefix = '/content/gdrive/My Drive/Tubes_1301164036/'
driveSuffix = 'ml-latest-small/tags.csv'
df_tag = pd.read_csv(drivePrefix + driveSuffix, sep=',')

# n_users = df.user_id.unique().shape[0]
n_items = df_tag.userId.unique().shape[0]
print('number of user = ' + str(n_items))
print(df_tag)

number of user = 58
      userId  movieId               tag   timestamp
0          2    60756             funny  1445714994
1          2    60756   Highly quotable  1445714996
2          2    60756      will ferrell  1445714992
3          2    89774      Boxing story  1445715207
4          2    89774               MMA  1445715200
...      ...      ...               ...         ...
3678     606     7382         for katie  1171234019
3679     606     7936           austere  1173392334
3680     610     3265            gun fu  1493843984
3681     610     3265  heroic bloodshed  1493843978
3682     610   168248  Heroic Bloodshed  1493844270

[3683 rows x 4 columns]


In [0]:
tag = []
for i in df_tag.tag.unique():
  tag.append(str.lower(i))
tag = np.unique(tag)

In [0]:
for i in range (len(df_tag.tag)):
  df_tag.tag[i] = str.lower(df_tag.tag[i])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [0]:
df_tag.tag.unique()

array(['funny', 'highly quotable', 'will ferrell', ..., 'austere',
       'gun fu', 'heroic bloodshed'], dtype=object)

In [0]:
vec_tag = pd.DataFrame(index =df_tag.movieId.unique(), columns = df_tag.tag.unique()).fillna(0)

In [0]:
for i in range (len(df_tag)):
   vec_tag.loc[df_tag.movieId.unique()==df_tag.movieId[i], vec_tag.keys()==str.lower(df_tag.tag[i])] = 1

In [0]:
vec_tag.keys()==str.lower(df_tag.tag[0])

array([ True, False, False, ..., False, False, False])

In [0]:
vec_tag

,funny,highly quotable,will ferrell,boxing story,mma,tom hardy,drugs,leonardo dicaprio,martin scorsese,way too long,al pacino,gangster,mafia,holocaust,true story,twist ending,anthony hopkins,courtroom drama,britpop,indie record label,music,dumpster diving,sustainability,romantic comedy,wedding,painter,bloody,black hole,sci-fi,time-travel,fantasy,magic board game,robin williams,beautiful scenery,epic,historical,inspirational,medieval,mel gibson,oscar (best cinematography),...,dull,oscar (best effects - visual effects),relaxing,revolutionary,setting:space/space ship,slow,slow paced,spacecraft,superb soundtrack,technology,tedious,visual,bowling,deadpan,john goodman,julianne moore,marijuana,philip seymour hoffman,ransom,rug,satirical,challenging,chuck palahniuk,consumerism,david fincher,double life,fighting,imaginary friend,mind-blowing,nudity (topless),palahnuik,postmodern,societal criticism,british,romans,70mm,for katie,austere,gun fu,heroic bloodshed
60756,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
89774,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
106782,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
48516,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
431,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1948,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
5694,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
6107,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7936,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [0]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=50,random_state=0).fit(vec_tag)

In [0]:
len(np.unique(kmeans.labels_))

50

In [0]:
np.unique(kmeans.labels_)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49],
      dtype=int32)

In [0]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = [0 for i in range(len(np.unique(kmeans.labels_)))]
for i in range(len(np.unique(kmeans.labels_))):  
  idx2 = np.where(kmeans.labels_==i)
  idx_cos = df_tag.movieId.unique()[kmeans.labels_==i]
  cosine_sim[i] = cosine_similarity(vec_tag.iloc[idx2],vec_tag.iloc[idx2])
  cosine_sim[i] = pd.DataFrame(cosine_sim[i], index=idx_cos,columns=idx_cos)

In [0]:
cosine_sim[2]

,4223,1272,2028,5316,6216,6235,7479,8446,6107
4223,1.000000,0.447214,0.447214,0.447214,0.447214,0.316228,0.447214,0.447214,0.447214
1272,0.447214,1.000000,1.000000,1.000000,1.000000,0.707107,1.000000,1.000000,1.000000
2028,0.447214,1.000000,1.000000,1.000000,1.000000,0.707107,1.000000,1.000000,1.000000
5316,0.447214,1.000000,1.000000,1.000000,1.000000,0.707107,1.000000,1.000000,1.000000
6216,0.447214,1.000000,1.000000,1.000000,1.000000,0.707107,1.000000,1.000000,1.000000
6235,0.316228,0.707107,0.707107,0.707107,0.707107,1.000000,0.707107,0.707107,0.707107
7479,0.447214,1.000000,1.000000,1.000000,1.000000,0.707107,1.000000,1.000000,1.000000
8446,0.447214,1.000000,1.000000,1.000000,1.000000,0.707107,1.000000,1.000000,1.000000
6107,0.447214,1.000000,1.000000,1.000000,1.000000,0.707107,1.000000,1.000000,1.000000


In [0]:
recommend_cluster = []
for data in range(len(R1)) :
  temp = np.where(R1.iloc[data]>=4.0)
  temp = pd.Series(R1.iloc[data]).keys()[temp]
  predict_recommend,idx_top5,temp2 = [],[],[]
  for film in temp:
    if film in df_tag.movieId.unique():
      pencarian = kmeans.labels_[df_tag.movieId.unique()==film] 
      top_5 = pd.Series(cosine_sim[pencarian[0]][film]).sort_values(ascending = False).keys()[1:6]
      predict_recommend.append(set(top_5))
  if len(predict_recommend) != 0:
    temp2 = list(set.union(*predict_recommend))
  recommend_cluster.append(temp2)

In [0]:
pd.DataFrame(recommend_cluster)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,722,723,724,725,726
0,516.0,1029.0,1030.0,1032.0,2058.0,11.0,6156.0,14.0,1042.0,2070.0,29.0,4639.0,8228.0,1573.0,122918.0,3111.0,41.0,43.0,556.0,1580.0,47.0,5682.0,2616.0,6201.0,1082.0,27706.0,62.0,1088.0,1090.0,2115.0,1096.0,6216.0,2121.0,2122.0,3147.0,589.0,592.0,8784.0,594.0,595.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,143367.0,6281.0,61323.0,27020.0,7438.0,5266.0,147.0,1171.0,1042.0,2070.0,155288.0,153.0,1304.0,1179.0,413.0,1953.0,1059.0,36517.0,1834.0,122922.0,6188.0,556.0,8375.0,50872.0,27831.0,183611.0,85565.0,83134.0,116797.0,27838.0,3265.0,62.0,57669.0,8645.0,3911.0,5064.0,179401.0,970.0,8011.0,1357.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6660.0,1029.0,1030.0,1032.0,1545.0,11.0,6156.0,14.0,8207.0,6159.0,1042.0,2579.0,2070.0,22.0,3097.0,29.0,4639.0,4641.0,7713.0,6181.0,41.0,7212.0,556.0,47.0,5682.0,6201.0,58.0,1082.0,62.0,1088.0,2115.0,1096.0,2121.0,2122.0,3147.0,8784.0,594.0,595.0,596.0,1625.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,108932.0,2312.0,648.0,3338.0,1032.0,7438.0,8207.0,7440.0,89745.0,98961.0,21.0,930.0,8228.0,122918.0,2726.0,2470.0,2471.0,3250.0,2355.0,5682.0,58.0,3259.0,27838.0,4160.0,3265.0,4034.0,5446.0,1223.0,1224.0,1225.0,457.0,1096.0,1235.0,1230.0,1231.0,1233.0,1234.0,8530.0,594.0,1237.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605,3072.0,1029.0,1030.0,143367.0,1032.0,2054.0,80906.0,11.0,6156.0,27660.0,6157.0,8207.0,14.0,6159.0,1042.0,21.0,2070.0,22.0,3097.0,26.0,3098.0,2076.0,29.0,3101.0,2078.0,2080.0,6178.0,1059.0,8228.0,6181.0,36.0,3111.0,26662.0,41.0,1066.0,4138.0,7212.0,43.0,46.0,47.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
606,3072.0,516.0,11.0,14.0,2579.0,25.0,82459.0,4638.0,122918.0,3111.0,4138.0,1580.0,556.0,47.0,5682.0,1082.0,1084.0,4160.0,2115.0,4164.0,1096.0,2121.0,2122.0,3147.0,6216.0,589.0,8784.0,1625.0,96861.0,608.0,3683.0,2660.0,34405.0,3175.0,3178.0,1135.0,2160.0,116.0,4725.0,4214.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,1.0,122882.0,1029.0,1030.0,143367.0,2054.0,4105.0,2058.0,2571.0,27660.0,6157.0,1042.0,2579.0,2070.0,2243.0,82459.0,540.0,193565.0,82461.0,2078.0,2080.0,8228.0,2085.0,3111.0,74791.0,4138.0,1580.0,6188.0,47.0,7217.0,5682.0,66097.0,6711.0,2616.0,68157.0,62.0,116797.0,85565.0,3135.0,2115.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

#Hybrid MF dengan CB

In [0]:
def hapus(a,b):
  hasil_hapus = [ idx for idx in b if idx not in a]
  return hasil_hapus

def bobot(i):
  irisan_mf_tag = set.intersection(set(recommend[i]), set(recommend_cluster[i]))
  irisan_mf_genre = set.intersection(set(recommend[i]), set(recommend_cb[i])) 
  gabungan = list(set.union(irisan_mf_tag,irisan_mf_genre))
  bobot_kedua = list(hasil.iloc[i][gabungan].sort_values(ascending=False).keys()[:(20-len(irisan_3_metode))])
  return bobot_kedua

def cluster(i,a):
  irisan_cluster = list(set.intersection(set(recommend_cb[i]), set(recommend_cluster[i])))
  fix = list(hasil.iloc[i][irisan_cluster].sort_values(ascending=False).keys()[:(20-len(a))])
  fix = hapus(a,fix)
  return fix

def hasil_rat(i):
  hsl = hapus(bobot_ketiga,recommend[i])
  hsl = hsl[:(20- len(bobot_ketiga))]
  return hsl

In [0]:
hasil_mf_cb,coba = [],[]
for i in range (len(recommend)):
  irisan_3_metode = list(set.intersection(set(recommend[i]), set(recommend_cb[i]), set(recommend_cluster[i])))
  if (len(irisan_3_metode) <= 20):
    bobot_kedua  = bobot(i)
    bobot_kedua  = hapus(irisan_3_metode,bobot_kedua)
    hasil_gabungan = irisan_3_metode+bobot_kedua
    if(len(hasil_gabungan)<20):
      bobot_ketiga = cluster(i,hasil_gabungan)
      bobot_ketiga = hasil_gabungan+bobot_ketiga
      if(len(bobot_ketiga)<20):
        bobot_keempat = hasil_rat(i)
        hasil_mf_cb.append(bobot_ketiga+bobot_keempat)
      else:
        hasil_mf_cb.append(bobot_ketiga )
    else : 
      hasil_mf_cb.append(hasil_gabungan)
      # if (len(irisan_3_metode) == 0):
      #   hasil_mf_cb.append(bobot_kedua)
      # else :
      #   hasil_mf_cb.append(irisan_3_metode+bobot_kedua)
  else :
     hasil_mf_cb.append(irisan_3_metode)

In [0]:
pd.DataFrame(hasil_mf_cb)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,1198,166183,173619,8580,7122,124404,26840,6442,6021,6201,3086,74342,5867,41828,141818,26985,44889,85736,112940,56060
1,5867,44889,38304,26736,41724,2905,5008,1235,58559,74510,57669,92475,2138,166183,158882,1349,136556,69211,5537,138835
2,164367,5746,141718,2196,5468,174053,26985,5919,5764,5039,166183,158882,1349,136556,69211,5537,138835,27704,93008,2075
3,930,908,1198,3086,6201,140627,6021,6442,67618,8580,173619,38304,1178,3266,148671,141844,70708,41724,105540,191005
4,67618,112940,33893,475,5901,1233,5747,1096,8207,4011,594,595,166183,158882,1349,136556,69211,5537,138835,27704
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605,3266,930,174053,904,908,1198,5008,44889,3086,26587,150554,6201,26849,5468,4495,140627,3687,6021,134095,6442
606,1249,67618,26840,6442,4495,5867,141818,26985,1337,112940,4645,5901,1209,72554,121469,191005,5747,1950,2905,1233
607,44889,173619,7815,67618,141718,124404,2196,6442,6021,3687,27751,174053,106642,5867,41828,148881,141818,26985,5657,85736
608,112940,56060,5901,3003,7156,5747,1208,1178,3266,3147,166183,158882,1349,136556,69211,5537,138835,27704,93008,2075


In [0]:
print("Enter your userid : ")
x = input()
rec = []
for i in hasil_mf_cb[int(x)-1]:
  rec.append(list(df_genre.loc[df_genre['movieId'] == i].title))
# a = list(hasil.iloc[0][recommend[0].intersection(recommend_cb[0])])
print('recommendation movie for user ',x,' : ')
for i in range(20): 
  print('{0}:{1}'.format(i+1, rec[i][0]))

Enter your userid : 
1
recommendation movie for user  1  : 
1:Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)
2:Junior and Karlson (1968)
3:Fugitives (1986)
4:Into the Woods (1991)
5:King of Hearts (1966)
6:Snowflake, the White Gorilla (2011)
7:Sonatine (Sonachine) (1993)
8:Belle époque (1992)
9:American Friend, The (Amerikanische Freund, Der) (1977)
10:Lady Jane (1986)
11:Babes in Toyland (1934)
12:Captain Newman, M.D. (1963)
13:Thief (1981)
14:Don't Look Now: We're Being Shot At (La grande vadrouille) (1966)
15:Ordinary Miracle (1978)
16:Nirvana (1997)
17:Reefer Madness: The Movie Musical (2005)
18:BURN-E (2008)
19:A Most Wanted Man (2014)
20:I Served the King of England (Obsluhoval jsem anglického krále) (2006)
